In [2]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 31: Likelihood Ratio Tests

Recall maximum likelihood estimators. These are obtained by maximizing the likelihood function with respect to $\theta$, the parameter of interest. Let's go through an example:

### Example 1: Poisson Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the Poisson distribution with unknown parameter $\lambda$. Find $\hat{\lambda}_{ML}$, the maximum likelihood estimate of $\lambda$. 

We recall from Lesson 13 that the pmf of a Poisson distribution with unknown parameter $\lambda$ is 

$f(x;\lambda) = {\frac{e^{-\lambda} \lambda^x}{x!}}$

Here, we will skip straight to the log-likelihood function.

$l(\lambda \mid\textbf{x})= \sum_{i=1}^n \log f(x_i;\lambda) = \sum_{i=1}^n \log {\frac{e^{-\lambda} \lambda^x}{x!}}$

Using the natural log in place of the arbitrary logarithm operator, we get

$\sum_{i=1}^n \ln {\frac{e^{-\lambda} \lambda^x}{x!}} = \sum_{i=1}^n [\ln e^{-\lambda} \lambda^{x_i} - \ln x_i!]$

$ = \sum_{i=1}^n [\ln e^{-\lambda} + \ln \lambda^{x_i} - \ln x_i!]$

$ = \sum_{i=1}^n [-\lambda + x_i \ln \lambda - \ln x_i!]$

Taking the derivative and setting it equal to zero, we find

$\frac{dl}{d\lambda} = \sum_{i=1}^n (-1 + \frac{x_i}{\lambda}) = 0$

$-n + \frac{{n \bar x}}{\lambda} = 0$

Thus, we conclude that 

$\hat \lambda_{ML} = \bar x$

This makes intuitive sense because in a Poisson distribution, $\lambda$ represents the average number of counts in a specified amount of time.

### Likelihood Ratio Tests

Assume you are testing a hypothesis:
$$
H_0: \theta=\theta_0
$$
$$
H_1: \theta\neq \theta_0
$$

The idea behind a likelihood ratio test is to compare the likelihood of the hypothesized value ($L(\theta_0\mid\textbf{x})$) to the maximum likelihood given the data ($L(\hat{\theta}_{ML}\mid\textbf{x})$). If the hypothesized value of $\theta$ were feasible, the likelihood under $\theta_0$ should be close to the max. If the hypothesized value of $\theta$ were not feasible, $L(\theta_0\mid\textbf{x})$ should be much smaller. To make the comparison, we consider the likelihood ratio test statistic, $\Lambda$: 

$$
\Lambda=\frac{L(\theta_0\mid\textbf{x})}{L(\hat{\theta}_{ML}\mid\textbf{x})}
$$

Because $\hat{\theta}_{ML}$ is the maximum likelihood estimator, this ratio has a maximum value of 1. Large values of $\Lambda$ (close to 1) indicate that $\theta_0$ is feasible (lack of evidence to reject). Small values of $\Lambda$ (close to 0) indicate $\theta_0$ is not feasible (evidence to reject). 

But how close to 0 is "close"? 

To evaluate this, we will take advantage of a helpful result. It turns out that if the null hypothesis were true, $-2\log \Lambda$ approximately follows the chi-squared distribution with 1 degree of freedom. The proof is outside the scope of this class. 

[We have not yet talked about the chi-squared distribution. To learn more, consult scipy help (`scipy.stats.chi2`). This distribution has one parameter that we care about: degrees of freedom, referenced in scipy as `df`. Bottom line, a random variable that has a chi-squared distribution with `df` degrees of freedom has a domain of $[0,\infty)$ and an expected value of `df`.]

$$
-2\log \Lambda = 2\left[l(\hat{\theta}_{ML}\mid \textbf{x})-l(\theta_0\mid \textbf{x})\right] \approx \textsf{Chisq}(1)
$$

It is a little bit more intuitive to consider $-2\log \Lambda$; large values of this test statistic indicate evidence to reject the null, which is consistent with most other hypothesis tests we've looked at. 

### Example 2: Likelihood ratio test on $\lambda$ from Poisson distribution

(Example taken from Pruim 2011). An instructor believes the the number of students who arrive late for class follows a Poisson process with an average of 1 late arrival per lesson. In 10 consecutive lessons, he found the following number of late arrivals: (1,1,0,4,2,1,3,0,0,2). Conduct a likelihood ratio test on the following hypothesis:

$$
H_0: \lambda = 1
$$
$$
H_1: \lambda \neq 1
$$

We look to consider test statistic, $\Lambda$, in order to simplify the calculation with a chi-squared approximation. We will start with the log-likelihood function from example 1.

$l(\lambda \mid\textbf{x})= \sum_{i=1}^n \log f(x_i;\lambda) = \sum_{i=1}^n \log {\frac{e^{-\lambda} \lambda^x}{x!}} = \log {\frac{e^{-n\lambda}\lambda^{\sum x_i}}{\prod x_i!}}$

We then implement the relation given above for $\hat \lambda_{ML} = \bar x$  and $\lambda_o = 1$, substituting the log-likelihood function into the expression.

$-2\log \Lambda = 2\left[l(\hat{\theta}_{ML}\mid \textbf{x})-l(\theta_0\mid \textbf{x})\right]$

$-2\log \Lambda = 2\left[{\sum_{i=1}^n \log \bar x}-{n \bar x}-{\log \prod_{i=1}^n x_i!}-(-n-{\log \prod_{i=1}^n x_i!})\right]$

$-2\log \Lambda = 2\left[{\sum_{i=1}^n \log \bar x}-{n \bar x}+n\right]$

From here, we are able to proceed with our approximation method, which will give us a p-value for the likelihood ratio test.

In [7]:
# Data set
late_arrivals = [1,1,0,4,2,1,3,0,0,2] 

# Maximum likelihood estimate for lambda based on Example 1 (x bar)
lam_ML = np.mean(late_arrivals) 
xbar = lam_ML

# Chi-squared input from above
test_stat = 2*(log(xbar)*sum(late_arrivals)-10*xbar+10)

# See scipy documentation
test_stat2 = 1-stats.chi2.cdf(test_stat,df=1)
test_stat2

0.23320226974499014

From the description above, a high value for 1 minus this test statistic indicates evidence to reject the null hypothesis. Since we have restructured it, we are looking for a low value below 0.05. We do not have this, and therefore are unable to to reject $H_o$.

#### Alternative Method

In this method, I simply use the sample mean as the test statistic. The observed value was 1.4. In the below, I simulate under the hypothesized $\lambda$, 1, and determine how often the sample mean was further away from 1 as was 1.4. 

In [6]:
ts2=[np.mean(stats.poisson.rvs(1,size=10)) for _ in np.arange(10000)]
np.mean(ts2>=np.mean(late_arrivals))+np.mean(ts2<=(1-(np.mean(late_arrivals)-1)))

0.2754